## Load libraries and modules

In [1]:
#If you are using Google Colaboratory to run this code, please upload the whole folder to your Google Drive, and run this cell install the requirements.

#connect to the google drive if you use Google Colaboratory
from google.colab import drive
drive.mount('/content/drive')
#install the libraries
!pip install rasterio
!pip install geopandas

Mounted at /content/drive
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 19.3 MB 462 kB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.0 MB 5.1 MB/s 
     |████████████████████████████████| 16.7 MB 58.7 MB/s 
     |████████████████████████████████| 6.3 MB 54.7 MB/s 


In [2]:
import tensorflow as tf
import tensorflow.keras
%matplotlib inline
%pylab inline
pylab.rcParams['figure.figsize'] = (15, 10)
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from skimage import exposure

from tensorflow.keras.models import *
from tensorflow.keras.layers import *

from tensorflow.keras import backend as K
from tensorflow.keras.optimizers import SGD, Adadelta
from tensorflow.keras import regularizers

import os
import rasterio 
#import rasterio.warp             # Reproject raster samples
from rasterio import windows
#import geopandas as gps
#import PIL.Image
#import PIL.ImageDraw
import fiona 
import imageio
from osgeo import gdal

from shapely.geometry import Point, MultiPoint
from shapely.geometry import mapping, shape
import numpy as np               # numerical array manipulation
#from tqdm import tqdm
import cv2
import random
from rasterio.windows import Window
from skimage import measure
from sklearn.cluster import KMeans
from fiona.crs import from_epsg

%matplotlib inline
%pylab inline
pylab.rcParams['figure.figsize'] = (15, 10)
import matplotlib.pyplot as plt
import gc

!pip install ipython-autotime
%load_ext autotime

Populating the interactive namespace from numpy and matplotlib
Populating the interactive namespace from numpy and matplotlib


/usr/local/lib/python3.7/dist-packages/IPython/core/magics/pylab.py:161: UserWarning: pylab import has clobbered these variables: ['maximum', 'dot', 'add', 'average', 'multiply', 'subtract', 'minimum', 'shape', 'concatenate', 'random']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
time: 166 µs (started: 2022-08-12 10:49:14 +00:00)


In [3]:
#set the sys path where the modules locates
import sys
sys.path.insert(0,"code/core")

#If you are using Google Colaboratory, modify the path here
sys.path.insert(0,"/content/drive/MyDrive/Colab/Wildebeest-UNet/core")

#import modules
import data_generator
from data_generator import DataGenerator

import data_augmentation
from data_augmentation import *

import counting
from counting import *

import visualization
from visualization import *

import importlib


import tensorflow as tf
tf.config.list_physical_devices('GPU')

def gridwise_sample2(imgarray, patchsize):
    """Extract sample patches of size patchsize x patchsize from an image (imgarray) in a gridwise manner.
    """
    nrows, ncols, nbands = imgarray.shape
    patchsamples = np.zeros(shape=(0, patchsize, patchsize, nbands),
                            dtype=imgarray.dtype)
    for i in range(int(nrows/patchsize)):
        for j in range(int(ncols/patchsize)):
            tocat = imgarray[i*patchsize:(i+1)*patchsize,
                             j*patchsize:(j+1)*patchsize, :]
            tocat = np.expand_dims(tocat, axis=0)
            patchsamples = np.concatenate((patchsamples, tocat),
                                          axis=0)
    return patchsamples

def linear_stretch(image):
    C = image.shape
    image2 = image.copy()
    for i in range(image2.shape[2]):
        p2, p98 = np.percentile(image2[:, :, i], (0.5, 99.5))
        image2[:, :, i] = exposure.rescale_intensity(image2[:, :, i],
                                                      in_range=(p2, p98))
        
    return image2


time: 2.55 s (started: 2022-08-12 10:49:14 +00:00)


## Load the whole satellite image

In [4]:
#Define File directories
Data_folder =  "/SampleData/"
#If you are using Google Colaboratory, modify the path here
Data_folder = "/content/drive/MyDrive/Colab/Wildebeest-UNet/SampleData"
Folder = "/content/drive/MyDrive/Colab/Wildebeest-UNet"

time: 1.23 ms (started: 2022-08-12 10:49:17 +00:00)


In [5]:
CLASSIFY_PATH = os.path.join(Data_folder, "data_2009Aug/classify")
img_name = 'GE1_20090811_sample'
index = 'GE1_20090811_sample'
image_dir = os.path.join(CLASSIFY_PATH, img_name+'.tif')

print(image_dir)

/content/drive/MyDrive/Colab/zijingwu-Satellite-based-monitoring-of-wildebeest/SampleData/data_2009Aug/classify/GE1_20090811_sample.tif
time: 8.51 ms (started: 2022-08-12 10:49:17 +00:00)


In [6]:
src = rasterio.open(image_dir)
print(src.meta['transform'])

| 0.43, 0.00, 718459.13|
| 0.00,-0.43, 9857063.71|
| 0.00, 0.00, 1.00|
time: 892 ms (started: 2022-08-12 11:04:19 +00:00)


In [ ]:
image_data=gdal.Open(image_dir)
bands = [image_data.GetRasterBand(i+1).ReadAsArray() for i in range(image_data.RasterCount)]
new_image = np.stack(bands, axis=2)
del bands

print(np.shape(new_image))

(10000, 10000, 4)
time: 5.07 s (started: 2022-07-27 15:18:09 +00:00)


In [ ]:
PATCHSIZE = 336
NBANDS = new_image.shape[-1]
classify_list = gridwise_sample2(new_image, PATCHSIZE)
print("There are %i number of training patches" % (classify_list.shape[0]))

There are 841 number of training patches
time: 44.3 s (started: 2022-07-27 15:18:14 +00:00)


In [ ]:
del new_image

numpy.save(os.path.join(CLASSIFY_PATH, 'list'+img_name+'.npy'), classify_list, allow_pickle=True, fix_imports=True)

time: 1.05 s (started: 2022-07-27 15:18:58 +00:00)


## Classify the whole image and detect the wildebeest

In [ ]:

# #after converting to 8int data type
# CLASSIFY_PATH = os.path.join(Data_folder, "data_2009Aug/classify/")
# img_name = 'GE1_20090811_sample'
# index = 'GE1_20090811_sample'
# image_dir = os.path.join(CLASSIFY_PATH, img_name+'.tif')

# print(image_dir)

classify_list = np.load(os.path.join(CLASSIFY_PATH, 'list'+img_name+'.npy'))

print(np.shape(classify_list))
print(np.max(classify_list))
# If the image is not in goog quality (low contrast), use the following function to add a linear contrast.
# for i in range(classify_list.shape[0]):
#     classify_list[i] = linear_stretch(classify_list[i])

# print(np.max(classify_list))

READ_CHANNEL = [1,2,3,4]
# READ_CHANNEL = [1,2,3]
NBANDS = len(READ_CHANNEL)
classify_list = classify_list[:,:,:,0:NBANDS]

print(type(classify_list))
print(np.shape(classify_list))


image_src=rasterio.open(image_dir)

nrows = np.int(image_src.shape[0]/336)
ncols = np.int(image_src.shape[1]/336)


# The size in pixels of your desired window
PATCHSIZE = 336
xsize, ysize = PATCHSIZE*ncols, PATCHSIZE*nrows

# Create a Window and calculate the transform from the source dataset    
window = Window(0, 0, xsize, ysize)
transform = image_src.window_transform(window)

# Create a new cropped raster to write to
profile = image_src.profile
profile.update({
    'height': ysize,
    'width': xsize,
    'transform': transform})

print(xsize, ysize)


(841, 336, 336, 4)
255
<class 'numpy.ndarray'>
(841, 336, 336, 4)
9744 9744
time: 797 ms (started: 2022-07-27 15:24:07 +00:00)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:31: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:32: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecat

In [ ]:

## Classify the patches
WEIGHT_PATH = os.path.join(Folder, 'tmp/pretrained_weights')

#    Edited by Sizhuo Li
#    Author: Ankit Kariryaa, University of Bremen
#Tversky loss function, which assigns more weights to false positive/false negative by adjusting alpha and beta parameters
def tversky(y_true, y_pred, alpha=0.1, beta=0.9):
    """
    Function to calculate the Tversky loss for imbalanced data
    :param prediction: the logits
    :param ground_truth: the segmentation ground_truth
    :param alpha: weight of false positives
    :param beta: weight of false negatives
    :param weight_map:
    :return: the loss
    """
    '''

    EPSILON = 0.00001
    '''
    y_true_pos = K.flatten(y_true)
    y_pred_pos = K.flatten(y_pred)
    # TP
    true_pos = K.sum(y_true_pos * y_pred_pos)
    # FN
    false_neg = K.sum(y_true_pos * (1-y_pred_pos))
    # FP
    false_pos = K.sum((1-y_true_pos) * y_pred_pos)
    return 1 - (true_pos + K.epsilon())/(true_pos + alpha * false_neg + beta * false_pos + K.epsilon())

def accuracy(y_true, y_pred, threshold=0.5):
    """compute accuracy"""
    y_t = y_true[...,0]
    y_t = y_t[...,np.newaxis]
    y_pred = K.round(y_pred +0.5 - threshold)
    return K.equal(K.round(y_t), K.round(y_pred))


# K.round() returns the Element-wise rounding to the closest integer!!!
# so the threshold to determine a true positive is here!!!!!
def true_positives(y_true, y_pred, threshold=0.5):
    """compute true positive"""
    #y_t = y_true[...,0]
    #y_t = y_t[...,np.newaxis]
    y_pred = K.round(y_pred +0.5 - threshold)
    return K.round(y_true * y_pred)

def false_positives(y_true, y_pred, threshold=0.5):
    """compute false positive"""
    #y_t = y_true[...,0]
    #y_t = y_t[...,np.newaxis]
    y_pred = K.round(y_pred +0.5 - threshold)
    return K.round((1 - y_true) * y_pred)

def true_negatives(y_true, y_pred, threshold=0.5):
    """compute true negative"""
    #y_t = y_true[...,0]
    #y_t = y_t[...,np.newaxis]
    y_pred = K.round(y_pred +0.5 - threshold)
    return K.round((1 - y_true) * (1 - y_pred))

def false_negatives(y_true, y_pred, threshold=0.5):
    """compute false negative"""
    #y_t = y_true[...,0]
    #y_t = y_t[...,np.newaxis]
    y_pred = K.round(y_pred +0.5 - threshold)
    return K.round((y_true) * (1 - y_pred))
def recall_m(y_true, y_pred):
    #y_t = y_true[...,0]
    #y_t = y_t[...,np.newaxis]
    tp = true_positives(y_true, y_pred)
    fn = false_negatives(y_true, y_pred)
    recall = K.sum(tp) / (K.sum(tp) + K.sum(fn)+ K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    #y_t = y_true[...,0]
    #y_t = y_t[...,np.newaxis]
    tp = true_positives(y_true, y_pred)
    fp = false_positives(y_true, y_pred)
    precision = K.sum(tp) / (K.sum(tp) + K.sum(fp)+ K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

#Model builder
#    Author: Ankit Kariryaa, University of Bremen
#!pip install tensorflow-estimatior==2.1.0
from tensorflow.keras.optimizers import Adam, Adadelta, Adagrad, Nadam

def unet(pretrained_weights = None,input_size = (PATCHSIZE,PATCHSIZE,NBANDS),lr = 1.0e-04, weight_b = 0.9, regularizers = regularizers.l2(0.0001)):
    inputs = Input(input_size)
    #conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(inputs)
    conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(inputs)
    conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv1)
    norm1 = BatchNormalization()(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(norm1)
    conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool1)
    conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv2)
    norm2 = BatchNormalization()(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(norm2)
    conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool2)
    conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv3)
    norm3 = BatchNormalization()(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(norm3)
    conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool3)
    conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv4)
    norm4 = BatchNormalization()(conv4)
    drop4 = Dropout(0.5)(norm4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)

    conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool4)
    conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv5)
    drop5 = Dropout(0.5)(conv5)

    up6 = Conv2D(512, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(drop5))
    norm6 = BatchNormalization()(up6)
    merge6 = Concatenate(axis = 3)([drop4, norm6])
    conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge6)
    conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv6)

    up7 = Conv2D(256, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv6))
    norm7 = BatchNormalization()(up7)
    merge7 = Concatenate(axis = 3)([norm3,norm7])
    conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge7)
    conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv7)

    up8 = Conv2D(128, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv7))
    norm8 = BatchNormalization()(up8)
    merge8 = Concatenate(axis = 3)([norm2,norm8])
    conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge8)
    conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv8)

    up9 = Conv2D(64, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv8))
    norm9 = BatchNormalization()(up9)
    merge9 = Concatenate(axis = 3)([norm1,norm9])
    conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge9)
    conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    conv9 = Conv2D(2, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    conv10 = Conv2D(1, 1, activation = 'sigmoid', kernel_regularizer= regularizers)(conv9)
    # set the probablity threshold that determines the wildebeest existence

    model = Model(inputs = inputs, outputs = conv10)

    #model.compile(optimizer = Adam(lr = 1e-4), loss =BinaryFocalLoss(gamma=2.0, pos_weight=0.25), metrics = ['accuracy', recall_m, precision_m, f1_m])
    
    #Tree count code
    OPTIMIZER = Adam(learning_rate = lr, beta_1= 0.9, beta_2= 0.999, epsilon= 1.0e-8)
    LOSS = tversky


    model.compile(optimizer=OPTIMIZER, loss=LOSS, metrics=[accuracy, precision_m, recall_m, f1_m])
    #model.summary()


    if(pretrained_weights):
    	model.load_weights(pretrained_weights)

    return model

time: 268 ms (started: 2022-07-27 15:24:15 +00:00)


In [ ]:
weight_set = 0.9
lr_set = 1e-4

num_folds = 10
Ypredict = [0]

for i in range(num_folds):
    model = unet(pretrained_weights=None, input_size=(PATCHSIZE,PATCHSIZE,NBANDS), lr = lr_set, weight_b = weight_set, regularizers = regularizers.l2(0.0001))
    best_path = os.path.join(WEIGHT_PATH, 'best_weights_fold_' + str(i+1) + '.hdf5')
    model.load_weights(best_path)
    predict = model.predict(classify_list)
    if np.max(predict) > 0.05:
      predict = (predict-np.min(predict))/(np.max(predict)-np.min(predict))
    print(np.shape(predict))
    Ypredict += predict
    del predict
  
    print(i+1)
    

predict = Ypredict/10
del Ypredict

print(np.max(predict))
print(np.min(predict))
predict[predict>0.5]=1.0
predict[predict<=0.5]=0.0
predict = predict.astype(np.int8)

print(predict.dtype)
print(np.shape(predict))


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


(841, 336, 336, 1)
1
(841, 336, 336, 1)
2
(841, 336, 336, 1)
3
(841, 336, 336, 1)
4
(841, 336, 336, 1)
5
(841, 336, 336, 1)
6
(841, 336, 336, 1)
7
(841, 336, 336, 1)
8
(841, 336, 336, 1)
9
(841, 336, 336, 1)
10
1.0
0.0
int8
(841, 336, 336, 1)
time: 7min 40s (started: 2022-07-27 15:24:17 +00:00)


In [ ]:
image_src=rasterio.open(image_dir)

nrows = np.int(image_src.shape[0]/336)
ncols = np.int(image_src.shape[1]/336)

# The size in pixels of your desired window
PATCHSIZE = 336
xsize, ysize = PATCHSIZE*ncols, PATCHSIZE*nrows

print(nrows, ncols, xsize, ysize)

classified_image = np.zeros(shape=(0, xsize, 1),dtype=predict.dtype)

for i in range(nrows):
  image_row = np.concatenate(predict[ncols*i:ncols*(i+1),:,:,:], axis = 1)
  classified_image = np.concatenate((classified_image, image_row), axis = 0)

print(np.shape(classified_image))
del predict


from osgeo import osr              ###
srs = osr.SpatialReference()       ###
srs.SetFromUserInput("EPSG:32736")  ###
# srs.SetFromUserInput("EPSG:4326")  ###
wgs84 = srs.ExportToProj4() 

#no use
profile = image_src.profile
profile.update({
    'crs': wgs84})

transform = image_src.meta['transform']
crs = image_src.meta['crs']

new_dataset = rasterio.open(os.path.join(CLASSIFY_PATH, index+'classifiedInt.tif'), 'w', driver='GTiff',
                            height = classified_image.shape[0], width = classified_image.shape[1],
                            count=1, dtype=str(classified_image.dtype),
                            crs=wgs84,
                            transform=transform)

new_dataset.write(classified_image[:,:,0],1)
new_dataset.close()

del image_row
del classified_image

from osgeo import osr              ###
srs = osr.SpatialReference()       ###
srs.SetFromUserInput("EPSG:32736")  ###
# srs.SetFromUserInput("EPSG:4326")  ###
wgs84 = srs.ExportToProj4() 

schema = {
    'geometry': 'Point',
    'properties': {'id': 'str'},
    }
def createShapefileObject(points, meta, wfile):
    #with fiona.open(wfile, 'w', crs=meta.get('crs').to_dict(), driver='ESRI Shapefile', schema=schema) as sink:
    with fiona.open(wfile, 'w', crs=wgs84, driver='ESRI Shapefile', schema=schema) as sink:
        for idx, point in enumerate(points):
            sink.write({
                'geometry': mapping(point),
                'properties': {'id': str(idx)},
                })
            #print(mapping(point))

classified_src = rasterio.open(os.path.join(CLASSIFY_PATH, index+'classifiedInt.tif'))
print(classified_src.meta)

image = classified_src.read() #shape of image is (1, rows, cols)
image = image[0,:,:]
all_points = ImageToPoints(image, classified_src)
print(len(all_points)) 
count = len(all_points)

createShapefileObject(all_points, classified_src.meta, os.path.join(CLASSIFY_PATH, index+'.shp'))

import pandas as pd
k = 1
print(count)
uncertainty = {}
uncertainty['ID'] = img_name
uncertainty['k'] = k
uncertainty['count'] = count
results = pd.DataFrame(uncertainty, index=[0])
results.to_csv(os.path.join(CLASSIFY_PATH, index+'_count.csv'))

29 29 9744 9744


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review yo

(9744, 9744, 1)
{'driver': 'GTiff', 'dtype': 'int8', 'nodata': None, 'width': 9744, 'height': 9744, 'count': 1, 'crs': CRS.from_epsg(4326), 'transform': Affine(0.4296371999999508, 0.0, 718459.127,
       0.0, -0.4296371999999508, 9857063.7052)}
41331
41331
time: 44.7 s (started: 2022-07-27 15:31:58 +00:00)


In [ ]:
print(count)

41331
time: 910 µs (started: 2022-07-27 15:32:43 +00:00)
